In [1]:
import os
import json
import pandas as pd
import numpy as np
import cvxpy as cp
import ast

### read in data

In [2]:
market_cap_categories = {
    'XL Market Cap': 5e10,
    'Large Market Cap': 1e10,
    'Medium Market Cap': 5e9,
    'Small Market Cap': 1e9
}

def define_marketcap_category(market_cap: float):
    for key, value in market_cap_categories.items():
        if market_cap > value:
            return key
    return 'XS Market Cap'

In [61]:
def process_price_data(path: str ='data/prices.csv', window_size: int = 7, max_null_price: int = 50):

    # read and pivot
    df_prices = pd.read_csv(path, parse_dates=['date'])
    pivot_price = df_prices.pivot(index='date', columns='coin', values='prices')

    # drop any with too many null values
    n_null_price = pivot_price.isna().sum()
    min_null_price = max_null_price
    to_drop = n_null_price[n_null_price > min_null_price].index.to_list()
    pivot_price = pivot_price[[c for c in pivot_price.columns if c not in to_drop]]
    
    # compute expected return and covariance
    pct_change = pivot_price.diff(window_size) / pivot_price
    mu_expected_return = pct_change.mean()
    sigma_covariance = pct_change.cov()
    return mu_expected_return, sigma_covariance, df_prices, to_drop

def process_coin_metadata(
        to_drop: list,
        metadata_path = 'data/coin_metadata.csv',
        category_groupings_path = 'data/category_groupings.json',
):
    
    with open(category_groupings_path, 'r', encoding='utf-8') as f:
        dct_category_groupings = json.load(f)
    lst_categories = sorted(set(np.concatenate([list(v) for v in dct_category_groupings.values()])))

    df_meta = pd.read_csv(metadata_path)
    df_meta = df_meta[~df_meta['id'].isin(to_drop)]
    df_meta['categories'] = df_meta['categories'].apply(ast.literal_eval)
    df_meta['market_cap_category'] = df_meta['market_caps'].map(define_marketcap_category)

    df_categories = df_meta[['id','categories']].explode('categories')

    dct_coin_category = pd.concat([
        df_categories[df_categories['categories'].isin(lst_categories)].groupby('categories')['id'].apply(list),
        df_meta.groupby('market_cap_category')['id'].apply(list)
    ])

    lst_assets = sorted(df_meta['id'])
    lst_categories = sorted(dct_coin_category.keys())

    return df_meta.set_index('id'), dct_category_groupings, dct_coin_category, lst_assets, lst_categories

In [100]:
with open('replace.json','w') as f :
    json.dump({key: sorted(val) for key, val in dct_category_groupings.items()},f)

In [97]:
mu_expected_return, sigma_covariance, df_prices, to_drop = process_price_data()

df_meta, dct_category_groupings, dct_coin_category, lst_assets, lst_categories = process_coin_metadata(to_drop=to_drop)


min_weights_assets = {asset: 0.0 for asset in lst_assets}
max_weights_assets = {asset: 0.5 for asset in lst_assets}

min_weights_categories = {category: 0.0 for category in lst_categories}
max_weights_categories = {category: 0.5 for category in lst_categories}

min_weights_categories = {
    grouping: {cat:0.0 for cat in categories}
    for grouping, categories in dct_category_groupings.items()
}

max_weights_categories = {
    grouping: {cat:1.0 for cat in categories}
    for grouping, categories in dct_category_groupings.items()
}


In [104]:
name_dict = df_meta['Name'].to_dict()


KeyError: 'Name'

In [108]:
dct_category_groupings['Ecosystem']

['Ethereum Ecosystem',
 'Avalanche Ecosystem',
 'BNB Chain Ecosystem',
 'Solana Ecosystem',
 'Polygon Ecosystem',
 'Near Protocol Ecosystem',
 'Moonriver Ecosystem',
 'Kava Ecosystem',
 'Tron Ecosystem',
 'Arbitrum Ecosystem',
 'Harmony Ecosystem',
 'Optimism Ecosystem',
 'Cosmos Ecosystem',
 'Gnosis Chain Ecosystem',
 'Fantom Ecosystem',
 'Polkadot Ecosystem',
 'Moonbeam Ecosystem',
 'Osmosis Ecosystem',
 'Cronos Ecosystem',
 'ZkSync Ecosystem',
 'Metis Ecosystem',
 'Velas Ecosystem',
 'Base Ecosystem',
 'Cardano Ecosystem',
 'Arbitrum Nova Ecosystem',
 'Ronin Ecosystem',
 'Aptos Ecosystem',
 'Curve Ecosystem']

In [118]:
eco = df_meta[['market_caps','categories']].explode('categories')
sorted(eco[eco['categories'].isin(dct_category_groupings['Ecosystem'])].groupby('categories')['market_caps'
                                                                                       ].sum().sort_values().tail(10).index.to_list())

['Arbitrum Ecosystem',
 'Avalanche Ecosystem',
 'BNB Chain Ecosystem',
 'Ethereum Ecosystem',
 'Kava Ecosystem',
 'Moonriver Ecosystem',
 'Near Protocol Ecosystem',
 'Polygon Ecosystem',
 'Solana Ecosystem',
 'Tron Ecosystem']

In [105]:
df_meta

,symbol,name,asset_platform_id,market_cap_rank,categories,market_caps,market_cap_category
id,,,,,,,
bitcoin,btc,Bitcoin,NaN,1,"[FTX Holdings, Cryptocurrency, Proof of Work (...",1.367669e+12,XL Market Cap
ethereum,eth,Ethereum,NaN,2,"[FTX Holdings, Proof of Stake (PoS), Multicoin...",4.492210e+11,XL Market Cap
tether,usdt,Tether,ethereum,3,"[FTX Holdings, Kava Ecosystem, Tron Ecosystem,...",1.033733e+11,XL Market Cap
binancecoin,bnb,BNB,NaN,4,"[Alleged SEC Securities, FTX Holdings, Proof o...",9.533214e+10,XL Market Cap
solana,sol,Solana,NaN,5,"[Alleged SEC Securities, FTX Holdings, Proof o...",8.184507e+10,XL Market Cap
...,...,...,...,...,...,...,...
icon,icx,ICON,NaN,250,[Smart Contract Platform],3.332206e+08,XS Market Cap
ontology,ont,Ontology,NaN,252,"[Proof of Stake (PoS), Masternodes, Smart Cont...",3.247287e+08,XS Market Cap
rollbit-coin,rlb,Rollbit Coin,ethereum,233,"[Gambling, Solana Ecosystem, Ethereum Ecosystem]",3.150623e+08,XS Market Cap


In [103]:
dct_coin_category['NFT Marketplace']

['blur', 'space-id', 'superfarm']

In [102]:
dct_coin_category['NFT']

['internet-computer',
 'immutable-x',
 'render-token',
 'theta-token',
 'fetch-ai',
 'floki',
 'gala',
 'flow',
 'axie-infinity',
 'the-sandbox',
 'apecoin',
 'chiliz',
 'decentraland',
 'ronin',
 'blur',
 'illuvium',
 'enjincoin',
 'ethereum-name-service',
 'space-id',
 'memecoin-2',
 'stepn',
 'superfarm',
 'project-galaxy',
 'audius']

In [94]:
min_weights_categories.keys()

dict_keys(['market_cap', 'portfolio', 'ecosystem', 'groupings_1', 'groupings_2'])

# basic

In [43]:
weights = cp.Variable(n_assets)
binary_selection = cp.Variable(n_assets, boolean=True)

mu = mu_expected_return.values
sigma = sigma_covariance.values
sigma_wrapped = cp.psd_wrap(sigma)

#max_risk = 0.05  # Maximum acceptable variance (risk level)
risk = cp.quad_form(weights, sigma_wrapped)

expected_return = mu.T @ weights
objective = cp.Maximize(expected_return)

constraints = [
    cp.sum(weights) == 1,  # Sum of weights is 1
    weights >= 0,          # No short selling
    #cp.sum(binary_selection) <= default_max_assets,  # No more than 5 assets
    #weights <= binary_selection,     # Link weights to selection
    weights >= [default_min_weights_assets[asset] for asset in lst_assets], #
    weights <= [default_max_weights_assets[asset] for asset in lst_assets], #
    #risk <= max_risk
]

prob = cp.Problem(objective, constraints)
prob.solve()

optimized_weights = weights.value

pd.Series(dict(zip(lst_assets, optimized_weights))).sort_values(ascending=False).head(10)

/Users/gm/repo/portfolio_opt/cryptocurrency_portfolio_optimiser/cryopt/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
/Users/gm/repo/portfolio_opt/cryptocurrency_portfolio_optimiser/cryopt/lib/python3.10/site-packages/cvxpy/atoms/quad_form.py:228: UserWarning: Forming a nonconvex expression quad_form(x, indefinite).
  warnings.warn("Forming a nonconvex expression quad_form(x, indefinite).")


nosana          5.000000e-01
bonk            5.000000e-01
aioz-network    1.881548e-12
paal-ai         1.207052e-12
mantra-dao      4.480167e-13
beam-2          4.215667e-13
superfarm       3.563607e-13
fetch-ai        3.071946e-13
corgiai         3.060157e-13
bittensor       2.976339e-13
dtype: float64

# more constraints

In [82]:
max_weights_categories['market_cap'] = {'XL Market Cap': 1.0,
  'Large Market Cap': 0.8,
  'Medium Market Cap': 0.5,
  'Small Market Cap': 0.2,
  'XS Market Cap': 0.0}

In [87]:
min_weights_categories['groupings_1'] = {
    'Layer 1 (L1)': 0.0,
    'Cryptocurrency': 0.2,
    'Smart Contract Platform': 0.3,
    'Stablecoins': 0.3,
    'Centralized Exchange (CEX)': 0.0,
    'Decentralized Finance (DeFi)': 0.0,
    'Meme': 0.0,
    'NFT': 0.0,
    'Layer 2 (L2)': 0.0,
    'NFT Marketplace': 0.0
    }

In [88]:
weights = cp.Variable(n_assets)
binary_selection = cp.Variable(n_assets, boolean=True)

mu = mu_expected_return.values
sigma = sigma_covariance.values
sigma_wrapped = cp.psd_wrap(sigma)

#max_risk = 0.05  # Maximum acceptable variance (risk level)
risk = cp.quad_form(weights, sigma_wrapped)

expected_return = mu.T @ weights
objective = cp.Maximize(expected_return)

constraints = [
    cp.sum(weights) == 1,  # Sum of weights is 1
    weights >= 0,          # No short selling
    #cp.sum(binary_selection) <= default_max_assets,  # No more than 5 assets
    #weights <= binary_selection,     # Link weights to selection
    weights >= [min_weights_assets[asset] for asset in lst_assets], #
    weights <= [max_weights_assets[asset] for asset in lst_assets], #
    #risk <= max_risk
]

for grouping, categories in dct_category_groupings.items():
    for category in categories:
        category_assets = dct_coin_category[category]
        category_indices = [i for i, asset in enumerate(lst_assets) if asset in category_assets]
        category_weight_sum = cp.sum(weights[category_indices])
        constraints += [
            category_weight_sum >= min_weights_categories[grouping][category],
            category_weight_sum <= max_weights_categories[grouping][category]
        ]

prob = cp.Problem(objective, constraints)
prob.solve()

optimized_weights = weights.value

allocation = pd.Series(dict(zip(lst_assets, optimized_weights))).sort_values(ascending=False)
allocation.head(20)

/Users/gm/repo/portfolio_opt/cryptocurrency_portfolio_optimiser/cryopt/lib/python3.10/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


solana               3.000000e-01
tether               2.999999e-01
bitcoin              2.000000e-01
bonk                 2.000000e-01
usd-coin             1.427607e-07
dogecoin             1.777598e-11
fetch-ai             1.226792e-11
near                 5.373671e-12
internet-computer    3.024110e-12
avalanche-2          2.351627e-12
uniswap              1.867087e-12
filecoin             1.600160e-12
litecoin             1.588652e-12
beam-2               1.481069e-12
chainlink            1.417076e-12
matic-network        1.280029e-12
ripple               1.268086e-12
cardano              1.236614e-12
staked-ether         1.084682e-12
binancecoin          1.081367e-12
dtype: float64

In [93]:
allocation_final = allocation.head(default_max_assets).round(3)
allocation_final = allocation_final[allocation_final > 0]
allocation_final = allocation_final * (1/allocation_final.sum())